<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/NRMS_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
path_file =  '/content/drive/MyDrive/KnowledgeGraph/Json_researcher/nrms.json'

In [4]:
f = open(path_file, "r",encoding="utf8")
count = 0
content = []
string_json = ''
begin = '['
end = ']'
split = ','

string_json += begin
while(True):
    # Read a line.
    line = f.readline()
    # When readline returns an empty string, the file is fully read.
    if line == "":
        print("::DONE::")
        break
    if count != 0:
      string_json += split
    string_json += line
    content.append(line)
    count +=1

string_json += end

::DONE::


In [5]:
import json
data = json.loads(string_json)

In [6]:
df = pd.json_normalize(data)

In [7]:
df

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01


# separate nrms

In [8]:
df.columns

Index(['_index', '_type', '_id', '_score', '_source.link', '_source.name',
       '_source.id', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', '_source.word', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate'],
      dtype='object')

In [9]:
def is_english(str):
    for i in str:
        if i in ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'):
            return True
    else:    
        return False

def is_thai(str):
    for i in str:
        if i in ('ก','ข','ฃ','ค','ฅ','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ'):
            return True
    else:
        return False

def language(text):
    if is_english(text):
      return 'eng'
    elif is_thai(text):
      return 'th'
    else:
      return 'other'

def strip_row(text):
    text = text.strip()
    text = ' '.join(text.split())
    return text

def first_name(list_text):
    name = ''
    if len(list_text) > 0:
      name = strip_row(list_text[0])
    return name

def check_first_author(row):
    if pd.isna(row['first_name']):
      return 'no'
    elif pd.isna(row['new_name']):
      return 'no'
    elif row['first_name'] == '':
      return 'no'
    elif row['first_name'] == row['new_name']:
      return 'yes'
    else:
      return 'no'

In [10]:
df_new_author = df.assign(new_name=df['_source.author'].str.split(","))

In [11]:
df_new_author

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[นายศิวกร อ่างทอง]
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[ผศ. อำนวย เรืองวารี]
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,[นายประสิทธิ์ ศรีนคร]
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[ผศ.ดร. ชัยยะ ปราณีตพลกรัง]
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[รศ. ดร. จตุรงค์ ลังกาพินธุ์]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายอดิศักดิ์ จำปาทอง]
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายบุญธง วสุริย์]
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[ผศ. อลิสา เลี้ยงรื่นรมย์]
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายนายสุรพล จักรชัยกุล]


In [12]:
df_new_author['first_name'] = df_new_author['new_name'].apply(first_name)
df_new_author = df_new_author.explode('new_name')

df_new_author = df_new_author.assign(new_name=df_new_author['new_name'].str.split(";"))
df_new_author = df_new_author.explode('new_name')

In [13]:
df_new_author

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล


In [14]:
df_new = pd.concat([df_new_author])

In [15]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล


In [16]:

df_new['new_name'] = df_new['new_name'].apply(strip_row)

In [17]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล


In [18]:
df_new['first_author'] = df_new.apply(check_first_author,axis=1) 

In [19]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,yes
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,yes
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,yes
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,yes
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,yes
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,yes
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,yes
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,yes


# clean name

In [20]:
def front(name, title):
    if name.find(title)>-1 and name.startswith(title):
        a = []
        b = ''
        for i in name:
            a.append(i)
        for i in range(len(title)):
            del a[0]
        for i in a:
            b = b+i
    return b 

def behind(name, title):
    if name.find(title)>-1 and name.endswith(title):
        a = []
        b = ''
        for i in name:
            a.append(i)
        for i in range(name.rfind(title),len(name)):
            del a[name.rfind(title)]
        for i in a:
            b = b+i
    return b

In [78]:
def deltitlefull(str):
    a = str
    b = []
    b.append(a)
    prefix = ''
    for i in b:
        if i in ('ว่าที่','จอมพล','พลเอก','พลโท','พลตรี','พลจัตวา','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอกพิเศษ','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','จอมพลเรือ','พลเรือเอก','พลเรือโท','พลเรือตรี','พลเรือจัตวา','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี','จอมพลอากาศ','พลอากาศเอก','พลอากาศโท','พลอากาศตรี','พลอากาศจัตวา','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท','เรืออากาศตรี','พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','พลตำรวจเอก','พลตำรวจโท','พลตำรวจตรี','พลตำรวจจัตวา','พันตำรวจเอก','พันตำรวจโท','พันตำรวจตรี','ร้อยตำรวจเอก','ร้อยตำรวจโท','ร้อยตำรวจตรี','ดาบตำรวจ','จ่าสิบตำรวจ','สิบตำรวจเอก','สิบตำรวจโท','สิบตำรวจตรี','หม่อมหลวง','ผู้ช่วยศาสตราจารย์','รองศาสตราจารย์','ศาสตราจารย์','อาจารย์','แพทย์หญิง','นายแพทย์','นาง','นางสาว','นาย'):
            return a, prefix
    c = ['ฯพณฯ']
    c1 = ['ผู้ช่วยศาสตราจารย์พิเศษ','รองศาสตราจารย์พิเศษ','ศาสตราจารย์พิเศษ','อาจารย์พิเศษ','ผู้ช่วยศาสตราจารย์','รองศาสตราจารย์','ศาสตราจารย์กิตติคุณ','ศาสตราจารย์เกียรติยศ','ศาสตราจารย์','อาจารย์']
    c2 = ['ว่าที่']
    c3 = ['จอมพลพิเศษหญิง','พลเอกพิเศษหญิง','พลโทพิเศษหญิง','พลตรีพิเศษหญิง','พลจัตวาพิเศษหญิง','พันเอกพิเศษหญิง','พันโทพิเศษหญิง','พันตรีพิเศษหญิง','ร้อยเอกพิเศษหญิง','ร้อยโทพิเศษหญิง','ร้อยตรีพิเศษหญิง','จ่าสิบเอกพิเศษพิเศษหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบโทพิเศษหญิง','จ่าสิบตรีพิเศษหญิง','สิบเอกพิเศษหญิง','สิบโทพิเศษหญิง','สิบตรีพิเศษหญิง','จอมพลเรือพิเศษหญิง','พลเรือเอกพิเศษหญิง','พลเรือโทพิเศษหญิง','พลเรือตรีพิเศษหญิง','พลเรือจัตวาพิเศษหญิง','นาวาเอกพิเศษหญิง','นาวาโทพิเศษหญิง','นาวาตรีพิเศษหญิง','เรือเอกพิเศษหญิง','เรือโทพิเศษหญิง','เรือตรีพิเศษหญิง','พันจ่าเอกพิเศษหญิง','พันจ่าโทพิเศษหญิง','พันจ่าตรีพิเศษหญิง','จ่าเอกพิเศษหญิง','จ่าโทพิเศษหญิง','จ่าตรีพิเศษหญิง','จอมพลอากาศพิเศษหญิง','พลอากาศเอกพิเศษหญิง','พลอากาศโทพิเศษหญิง','พลอากาศตรีพิเศษหญิง','พลอากาศจัตวาพิเศษหญิง','นาวาอากาศเอกพิเศษหญิง','นาวาอากาศโทพิเศษหญิง','นาวาอากาศตรีพิเศษหญิง','เรืออากาศเอกพิเศษหญิง','เรืออากาศโทพิเศษหญิง','เรืออากาศตรีพิเศษหญิง','พันจ่าอากาศเอกพิเศษหญิง','พันจ่าอากาศโทพิเศษหญิง','พันจ่าอากาศตรีพิเศษหญิง','จ่าอากาศเอกพิเศษหญิง','จ่าอากาศโทพิเศษหญิง','จ่าอากาศตรีพิเศษหญิง','พลตำรวจเอกพิเศษหญิง','พลตำรวจโทพิเศษหญิง','พลตำรวจตรีพิเศษหญิง','พลตำรวจจัตวาพิเศษหญิง','พันตำรวจเอกพิเศษหญิง','พันตำรวจโทพิเศษหญิง','พันตำรวจตรีพิเศษหญิง','ร้อยตำรวจเอกพิเศษหญิง','ร้อยตำรวจโทพิเศษหญิง','ร้อยตำรวจตรีพิเศษหญิง','ดาบตำรวจพิเศษหญิง','จ่าสิบตำรวจพิเศษหญิง','สิบตำรวจเอกพิเศษหญิง','สิบตำรวจโทพิเศษหญิง','สิบตำรวจตรีพิเศษหญิง','จอมพลพิเศษ','พลเอกพิเศษ','พลโทพิเศษ','พลตรีพิเศษ','พลจัตวาพิเศษ','พันเอกพิเศษ','พันโทพิเศษ','พันตรีพิเศษ','ร้อยเอกพิเศษ','ร้อยโทพิเศษ','ร้อยตรีพิเศษ','จ่าสิบเอกพิเศษพิเศษ','จ่าสิบเอกพิเศษ','จ่าสิบโทพิเศษ','จ่าสิบตรีพิเศษ','สิบเอกพิเศษ','สิบโทพิเศษ','สิบตรีพิเศษ','จอมพลเรือพิเศษ','พลเรือเอกพิเศษ','พลเรือโทพิเศษ','พลเรือตรีพิเศษ','พลเรือจัตวาพิเศษ','นาวาเอกพิเศษ','นาวาโทพิเศษ','นาวาตรีพิเศษ','เรือเอกพิเศษ','เรือโทพิเศษ','เรือตรีพิเศษ','พันจ่าเอกพิเศษ','พันจ่าโทพิเศษ','พันจ่าตรีพิเศษ','จ่าเอกพิเศษ','จ่าโทพิเศษ','จ่าตรีพิเศษ','จอมพลอากาศพิเศษ','พลอากาศเอกพิเศษ','พลอากาศโทพิเศษ','พลอากาศตรีพิเศษ','พลอากาศจัตวาพิเศษ','นาวาอากาศเอกพิเศษ','นาวาอากาศโทพิเศษ','นาวาอากาศตรีพิเศษ','เรืออากาศเอกพิเศษ','เรืออากาศโทพิเศษ','เรืออากาศตรีพิเศษ','พันจ่าอากาศเอกพิเศษ','พันจ่าอากาศโทพิเศษ','พันจ่าอากาศตรีพิเศษ','จ่าอากาศเอกพิเศษ','จ่าอากาศโทพิเศษ','จ่าอากาศตรีพิเศษ','พลตำรวจเอกพิเศษ','พลตำรวจโทพิเศษ','พลตำรวจตรีพิเศษ','พลตำรวจจัตวาพิเศษ','พันตำรวจเอกพิเศษ','พันตำรวจโทพิเศษ','พันตำรวจตรีพิเศษ','ร้อยตำรวจเอกพิเศษ','ร้อยตำรวจโทพิเศษ','ร้อยตำรวจตรีพิเศษ','ดาบตำรวจพิเศษ','จ่าสิบตำรวจพิเศษ','สิบตำรวจเอกพิเศษ','สิบตำรวจโทพิเศษ','สิบตำรวจตรีพิเศษ','จอมพลหญิง','พลเอกหญิง','พลโทหญิง','พลตรีหญิง','พลจัตวาหญิง','พันเอกหญิง','พันโทหญิง','พันตรีหญิง','ร้อยเอกหญิง','ร้อยโทหญิง','ร้อยตรีหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบเอกหญิง','จ่าสิบโทหญิง','จ่าสิบตรีหญิง','สิบเอกหญิง','สิบโทหญิง','สิบตรีหญิง','จอมพลเรือหญิง','พลเรือเอกหญิง','พลเรือโทหญิง','พลเรือตรีหญิง','พลเรือจัตวาหญิง','นาวาเอกหญิง','นาวาโทหญิง','นาวาตรีหญิง','เรือเอกหญิง','เรือโทหญิง','เรือตรีหญิง','พันจ่าเอกหญิง','พันจ่าโทหญิง','พันจ่าตรีหญิง','จ่าเอกหญิง','จ่าโทหญิง','จ่าตรีหญิง','จอมพลอากาศหญิง','พลอากาศเอกหญิง','พลอากาศโทหญิง','พลอากาศตรีหญิง','พลอากาศจัตวาหญิง','นาวาอากาศเอกหญิง','นาวาอากาศโทหญิง','นาวาอากาศตรีหญิง','เรืออากาศเอกหญิง','เรืออากาศโทหญิง','เรืออากาศตรีหญิง','พันจ่าอากาศเอกหญิง','พันจ่าอากาศโทหญิง','พันจ่าอากาศตรีหญิง','จ่าอากาศเอกหญิง','จ่าอากาศโทหญิง','จ่าอากาศตรีหญิง','พลตำรวจเอกหญิง','พลตำรวจโทหญิง','พลตำรวจตรีหญิง','พลตำรวจจัตวาหญิง','พันตำรวจเอกหญิง','พันตำรวจโทหญิง','พันตำรวจตรีหญิง','ร้อยตำรวจเอกหญิง','ร้อยตำรวจโทหญิง','ร้อยตำรวจตรีหญิง','ดาบตำรวจหญิง','จ่าสิบตำรวจหญิง','สิบตำรวจเอกหญิง','สิบตำรวจโทหญิง','สิบตำรวจตรีหญิง','จอมพล','พลเอก','พลโท','พลตรี','พลจัตวา','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอกพิเศษ','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','จอมพลเรือ','พลเรือเอก','พลเรือโท','พลเรือตรี','พลเรือจัตวา','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี','จอมพลอากาศ','พลอากาศเอก','พลอากาศโท','พลอากาศตรี','พลอากาศจัตวา','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท','เรืออากาศตรี','พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','พลตำรวจเอก','พลตำรวจโท','พลตำรวจตรี','พลตำรวจจัตวา','พันตำรวจเอก','พันตำรวจโท','พันตำรวจตรี','ร้อยตำรวจเอก','ร้อยตำรวจโท','ร้อยตำรวจตรี','ดาบตำรวจ','จ่าสิบตำรวจ','สิบตำรวจเอก','สิบตำรวจโท','สิบตำรวจตรี']   
    c4 = ['แพทย์หญิง','นายแพทย์','ทันตแพทย์หญิง','ทันตแพทย์','นางสาว','นาง','นาย']
    c5 = ['หม่อมหลวง','คุณหญิง','ท่านหญิง','ท่านผู้หญิง','หม่อมราชวงศ์หญิง','หม่อมราชวงศ์','หม่อมเจ้าหญิง','หม่อมเจ้า']


    for i in c:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c1:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c2:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c3:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c4:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c5:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    return a, prefix
   
def deltitleshort(str):
    a = str
    b = ''
    s  = ['พล.อ.','พล.ท.','พล.ต','พล.จ.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต.','ส.อ.','ส.ท.','ส.ต.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','พล.ร.จ.','น.อ.','น.ท.','น.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.','จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พล.อ.จ.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พล.ต.จ.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.','พญ.','นพ.','ดร.','น.ส.','ม.ล.','ม.ร.ว','ม.จ.','ผศ.','รศ.','ศ.','อ.']
    s1 = ('พญ.','นพ.','ดร.','น.ส.','น.สพ.','สพ.ญ.','ภญ.','ภกญ.')
    s2 = ('ผศ.','รศ.','ศ.','อ.')
    s3 = ('ม.ล.','ม.ร.ว','ม.จ.')
    for i in range(0,a.rfind('.')+1):
        b = b+a[i]

    prefix = ''
    for i in s:
        if b.endswith(i) == True:
            a = front(a,b)
            prefix = b
            for i in s1:
                if b.endswith(i) == True:
                    return a, prefix
            for i in s2:
                if b.endswith(i) == True:
                    if a.startswith('(พิเศษ)') == True:
                        a = front(a,'(พิเศษ)')
                        prefix += '(พิเศษ)'
                        return a, prefix
                    elif a.startswith('พิเศษ') == True:
                        a = front(a,'พิเศษ')
                        prefix += 'พิเศษ'
                        return a, prefix
                    else:
                        return a, prefix
            for i in s3:
                if b.endswith(i) == True:
                    if a.startswith('หญิง') == True:
                        a = front(a,'หญิง')
                        prefix += ('หญิง')
                        return a, prefix
                    elif a.startswith('(หญิง)') == True:
                        a = front(a,'(หญิง)')
                        prefix += '(หญิง)'
                        return a, prefix
            if a.startswith('(พิเศษ)') == True:
                a = front(a,'(พิเศษ)')
                prefix += '(พิเศษ)'
            elif a.startswith('พิเศษ') == True:
                a = front(a,'พิเศษ')
                prefix += 'พิเศษ'
            if a.startswith('หญิง') == True:
                a = front(a,'หญิง')
                prefix += 'หญิง'
            elif a.startswith('(หญิง)') == True:
                a = front(a,'(หญิง)')
                prefix += '(หญิง)'
            return a, prefix
    else:
        return a, prefix

def delnumber(str):
    p = ['0','1','2','3','4','5','6','7','8','9']
    a = []
    b = ''
    j = 0
    for i in str:
        a.append(i)
    while j < len(a):
        if a[j] in p:
            del a[j]
            j = j-1
        j = j+1   
    for i in a:
        b = b + i
    return b

def celanSpace(str):
    str = " ".join(str.split())
    return str.strip()

def delengtitle(str):
    a = str
    b= ''
    # c = ('Md.','MD.','md.','Mrs.','MRS.','mrs.','Ph.D.','Associate.','Assoc.','Asso.','Asst.','Assist.','Professor.','Prof.','Dr.','Mr.','Ms.','associate.','assoc.','asso.','asst.','assist.','professor.','prof.','dr.','mr.','ms.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','PROFESSOR.','PROF.','DR.','MR.','MS.',
    #      'Md ','MD ','md ','Mrs ','MRS ','mrs ','Ph.D ','Associate ','Assoc ','Asso ','Asst ','Assist ','Professor ','Prof ','Dr ','Mr ','Ms ','associate ','assoc ','asso ','asst ','assist ','professor ','prof ','dr ','mr ','ms ','ASSOCIATE ','ASSOC ','ASSO ','ASST ','ASSIST ','PROFESSOR ','PROF ','DR ','MR ','MS ')
    c = ('Associate.','Assoc.','Asso.','Asst.','Assist.','associate.','assoc.','asso.','asst.','assist.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','Associate ','Assoc ','Asso ','Asst ','Assist ','associate ','assoc ','asso ','asst ','assist ','ASSOCIATE ','ASSOC ','ASSO ','ASST ','ASSIST ',
          'Ph.D.','Professor.','Prof.','professor.','prof.','dr.','PROFESSOR.','PROF.','Ph.D ','Professor ','Prof ','professor ','prof ','PROFESSOR ','PROF ',
          'Dr.','DR.','Dr ','dr ','DR ',
          'Md.','MD.','md.','Md ','MD ','md ',
          'Mrs.','MRS.','mrs.','Mr.','Ms.','mr.','ms.','MR.','MS.','Mrs ','MRS ','mrs ','Mr ','Ms ','mr ','ms ','MR ','MS ')
    for i in range(0,a.rfind('.')+1):
        b += a[i]

    prefix = ''
    
    found = True
    while found:
      found = False
      for i in c:
          if a.find(i) != -1:
              # a = front(a,b)
              a = a.replace(i,"")
              found = True
              prefix = prefix + i
              break
    return a, prefix

In [79]:
delengtitle('Assist. Prof กิตติพงศ์ ไชยนอก	')

(' กิตติพงศ์ ไชยนอก\t', 'Assist.Prof ')

In [80]:
def delAllPrefix(str):
  str = delnumber(str)
  str, prefix1 = deltitlefull(str)
  str, prefix2 = deltitleshort(str)
  str, prefix3 = delengtitle(str)
  str = celanSpace(str)
  prefix = prefix1+prefix2+prefix3
  return str, prefix

def cleanName(row):
  clean_name, prefix = delAllPrefix(row['new_name'])
  clean_name = clean_name.replace(',','')
  row['clean_name'] = clean_name
  row['prefix'] = prefix
  return row

In [81]:
df_new = df_new.apply(cleanName,axis=1)

In [82]:
# df_new['language'] = df_new['new_name'].apply(language)
df_new['language'] = df_new['clean_name'].apply(language)

In [83]:
df_thai = df_new[df_new['language']=='th']
df_eng = df_new[df_new['language']=='eng']
df_other = df_new[df_new['language']=='other']

In [84]:
df_thai

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author,clean_name,prefix,language
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,yes,ศิวกร อ่างทอง,นาย,th
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,yes,อำนวย เรืองวารี,ผศ.,th
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,yes,ประสิทธิ์ ศรีนคร,นาย,th
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,yes,ชัยยะ ปราณีตพลกรัง,ผศ.ดร.,th
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,yes,จตุรงค์ ลังกาพินธุ์,รศ. ดร.,th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,yes,อดิศักดิ์ จำปาทอง,นาย,th
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,yes,บุญธง วสุริย์,นาย,th
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,yes,อลิสา เลี้ยงรื่นรมย์,ผศ.,th
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,yes,นายสุรพล จักรชัยกุล,นาย,th


In [85]:
df_eng

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author,clean_name,prefix,language
318,nrms,nrms,29419,1,,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,29419,Dr. Hien Van Doan,1/10/2561,30/9/2562,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",The main objective of the study is to investig...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Dr. Hien Van Doan,Dr. Hien Van Doan,yes,Hien Van Doan,Dr.,eng
962,nrms,nrms,2006,1,256109A3060066,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,2006,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,เปลี่ยนแปลงเล็กน้อย,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,i). estimate juvenile survivorship of understo...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,yes,Andrew Pierce,นาย,eng
2360,nrms,nrms,254239,1,2560A30402173,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,254239,Prof.Dr. James Ketudat-Cairns,1/10/2557,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",The purpose of this project is to identify and...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-24,Prof.Dr. James Ketudat-Cairns,Prof.Dr. James Ketudat-Cairns,yes,James Ketudat-Cairns,Prof.Dr.,eng
3799,nrms,nrms,232102,1,2559A12202062,การพัฒนาจิตอาสาโดยกระบวนการฟื้นฟูแหล่งเรียนรู้...,232102,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,1/10/2558,30/9/2559,โครงการ,ไม่เปลี่ยนแปลง,๔.๑ การพัฒนา หมายถึง การเจริญเติบโต (Growth) เ...,๖.๑ เพื่อฝึกอบรมเชิงปฏิบัติการให้นักเรียนเป็นเ...,",สาขาสังคมวิทยา",ผู้ประสานคณะ สถาบันสำนัก กำลังตรวจสอบ,0,มหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาลัย,2019-10-31,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,yes,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,,eng
4469,nrms,nrms,170318,1,2559A10402040,การออกแบบเจนเนอเรเตอร์ขนาดเมโส,170318,นายJames C. Moran,1/10/2558,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"เครื่องกำเนิดพลังงานขนาดเมโส,การเผาไหม้ขนาดเมโ...",1) Design and build a meso vaporization system...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยเชียงใหม่,2019-10-31,นายJames C. Moran,นายJames C. Moran,yes,James C. Moran,นาย,eng
4577,nrms,nrms,169897,1,2559A30602066,ผลกระทบของหย่อมป่าและพื้นที่ชายขอบป่าต่ออัตราก...,169897,นายAndrew Pierce,1/10/2558,30/9/2560,โครงการ,เปลี่ยนแปลงมาก,"การอนุรักษ์และจัดการความหลากหลายทางชีวภาพ,การล...",1. To determine the nesting patterns of birds ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-31,นายAndrew Pierce,นายAndrew Pierce,yes,Andrew Pierce,นาย,eng
4658,nrms,nrms,171182,1,2559A30402137,การจัดเรียงเพนตะควาร์กของบาริออนมวลเบาที่ไม่มี...,171182,นายYupeng Yan,1/10/2557,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ควาร์ก,มีซอน,บาริออน,โครงสร้างของบาริออน,เพนตะ...",To construct the wave function of three-quark ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-31,นายYupeng Yan,นายYupeng Yan,yes,Yupeng Yan,นาย,eng
5583,nrms,nrms,17515,1,,การจัดการตราสินค้าของแหล่งท่องเที่ยวเพื่อส่งเส...,17515,Dr. ChaiChing Tan,1/10/2561,30/9/2562,โครงการ,,การจัดการตราสินค้า/การท่องเที่ยวโดยชุมชน/การท่...,1. ศึกษาการรับรู้ของนักท่องเที่ยวด้านตราสินค้า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยแม่ฟ้าหลวง,2019-10-24,Dr. ChaiChing Tan,Dr. ChaiChing Tan,yes,ChaiChing Tan,Dr.,eng
5769,nrms,nrms,35608,1,,บทบาทของโครงสร้างป่าต่ออัตราการล่ารังนกโดยงูเข...,35608,Mr. George A. Gale,1/10/2561,30/9/2563,โครงการ,,.,The objectives are to understand how vegetatio...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,Mr. George A. Gale,Mr. George A. Gale,yes,George A.

In [86]:
df_other

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author,clean_name,prefix,language
46,nrms,nrms,11775,1,,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,11775,,1/10/2561,30/9/2563,โครงการ,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,Training and workshop to increase TB research ...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,no,,,other
78,nrms,nrms,12698,1,,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,12698,,1/10/2560,30/9/2562,โครงการ,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,เพื่อพัฒนาชุดควบคุมการปรับระดับเมนิสคัสแบบอัตโ...,,โครงการอยู่ระหว่างดำเนินการ,0,กรมวิทยาศาสตร์บริการ,2019-10-24,,,no,,,other
95,nrms,nrms,16150,1,,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,16150,,1/10/2561,30/9/2563,โครงการ,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",1) เพื่อประเมินโครงสร้างพื้นฐานของห้องปฏิบัติก...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,no,,,other
111,nrms,nrms,16384,1,,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,16384,,1/11/2561,30/7/2562,โครงการ,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,๑. เพื่อศึกษาสภาพปัญหาของมาตรการปฏิบัติต่อผู้ก...,,โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานกิจการยุติธรรม,2019-10-24,,,no,,,other
154,nrms,nrms,17505,1,,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,17505,,1/10/2561,30/9/2565,โครงการ,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",4.1 เพื่อพัฒนาเทคนิควิธีการทดสอบและสอบเทียบที่...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,สถาบันวิจัยวิทยาศาสตร์และเทคโนโลยีแห่งประเทศไทย,2019-10-24,,,no,,,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24312,nrms,nrms,5934,1,,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,5934,,,,โครงการ,โครงการที่เข้าใหม่,,เพื่อการพัฒนาการวิเคราะห์ปริมาณ C-14 ที่นำไปสู...,,โครงการอยู่ระหว่างดำเนินการ,0,สถาบันเทคโนโลยีนิวเคลียร์แห่งชาติ (องค์การมหาชน),2019-10-24,,,no,,,other
25500,nrms,nrms,271519,1,2560A00602003,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,271519,,1/10/2559,30/9/2560,โครงการ,ไม่เปลี่ยนแปลง,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",เพื่อพัฒนาการเพาะเลี้ยงปลากระแห เพื่อการอนุรัก...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานคณะกรรมการการอาชีวศึกษา,2019-10-31,,,no,,,other
27183,nrms,nrms,626770,1,2559NRCT72022,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,626770,,1/10/2558,30/9/2559,โครงการ,โครงการที่เข้าใหม่,,สำรวจ รวบรวมความหลากหลายทางชีวภาพของสายพันธุ์พ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการวิจัยเสร็จสิ้น,0,กองบริหารแผนและงบประมาณการวิจัย (กบง.),2019-11-01,,,no,,,other
27208,nrms,nrms,544939,1,,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,544939,,1/10/2559,31/1/2561,โครงการ,ไม่เปลี่ยนแปลง,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,เพื่อหนุนเสริมและเพิ่มขีดความสามารถของเกษตรกร ...,,โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,,,no,,,other


# create Author Table

In [87]:
uniq_name = df_new['clean_name'].unique()

In [30]:
uniq_name

array(['ศิวกร อ่างทอง', 'อำนวย เรืองวารี', 'ประสิทธิ์ ศรีนคร', ...,
       'นัทธีรัตน์ พีระพันธุ์', 'อดิศักดิ์ จำปาทอง', 'ฉลอง ทับศรี'],
      dtype=object)

In [31]:
df_unique_name = pd.DataFrame(uniq_name,columns=['clean_name'])

In [32]:
df_unique_name

,clean_name
0,ศิวกร อ่างทอง
1,อำนวย เรืองวารี
2,ประสิทธิ์ ศรีนคร
3,ชัยยะ ปราณีตพลกรัง
4,จตุรงค์ ลังกาพินธุ์
...,...
13653,จักรกฤษณ์ พจนศิลป์
13654,เติมเพชร สุขคณาภิบาล
13655,นัทธีรัตน์ พีระพันธุ์
13656,อดิศักดิ์ จำปาทอง


In [33]:
start_id = 528488
df_unique_name['id'] = df_unique_name.index + start_id

In [34]:
df_unique_name

,clean_name,id
0,ศิวกร อ่างทอง,528488
1,อำนวย เรืองวารี,528489
2,ประสิทธิ์ ศรีนคร,528490
3,ชัยยะ ปราณีตพลกรัง,528491
4,จตุรงค์ ลังกาพินธุ์,528492
...,...,...
13653,จักรกฤษณ์ พจนศิลป์,542141
13654,เติมเพชร สุขคณาภิบาล,542142
13655,นัทธีรัตน์ พีระพันธุ์,542143
13656,อดิศักดิ์ จำปาทอง,542144


In [35]:
result_thai = pd.merge(df_thai,
                       df_unique_name,
                       left_on='clean_name',
                       right_on='clean_name',
                       how='left')

result_eng = pd.merge(df_eng,
                       df_unique_name,
                       left_on='clean_name',
                       right_on='clean_name',
                       how='left')

result_other = pd.merge(df_other,
                       df_unique_name,
                       left_on='clean_name',
                       right_on='clean_name',
                       how='left')

result_all = pd.merge(df_new,
                       df_unique_name,
                       left_on='clean_name',
                       right_on='clean_name',
                       how='left')

In [36]:
result_thai

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author,clean_name,prefix,language,id
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,yes,ศิวกร อ่างทอง,นาย,th,528488
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,yes,อำนวย เรืองวารี,ผศ.,th,528489
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,yes,ประสิทธิ์ ศรีนคร,นาย,th,528490
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,yes,ชัยยะ ปราณีตพลกรัง,ผศ.ดร.,th,528491
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,yes,จตุรงค์ ลังกาพินธุ์,รศ. ดร.,th,528492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26557,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,yes,อดิศักดิ์ จำปาทอง,นาย,th,542144
26558,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,yes,บุญธง วสุริย์,นาย,th,530467
26559,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,yes,อลิสา เลี้ยงรื่นรมย์,ผศ.,th,535640
26560,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,

In [37]:
result_eng

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,first_author,clean_name,prefix,language,id
0,nrms,nrms,29416,1,,การใช้โปรไบโอติกที่พบในปลาเพื่อเป็นทางเลือกแทน...,29416,Assist. Prof. Dr.,1/10/2563,30/9/2564,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"โพรไบโอติก, ปลานิล, Streptococcus agalactiae",The main objective of this study is to investi...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Assist. Prof. Dr.,Assist. Prof. Dr.,yes,Assist. Prof.,Dr.,eng,528786
1,nrms,nrms,29419,1,,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,29419,Dr. Hien Van Doan,1/10/2561,30/9/2562,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",The main objective of the study is to investig...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Dr. Hien Van Doan,Dr. Hien Van Doan,yes,Hien Van Doan,Dr.,eng,528787
2,nrms,nrms,2006,1,256109A3060066,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,2006,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,เปลี่ยนแปลงเล็กน้อย,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,i). estimate juvenile survivorship of understo...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,yes,Andrew Pierce,นาย,eng,529341
3,nrms,nrms,254239,1,2560A30402173,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,254239,Prof.Dr. James Ketudat-Cairns,1/10/2557,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",The purpose of this project is to identify and...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-24,Prof.Dr. James Ketudat-Cairns,Prof.Dr. James Ketudat-Cairns,yes,Prof. James Ketudat-Cairns,Dr.,eng,530526
4,nrms,nrms,232102,1,2559A12202062,การพัฒนาจิตอาสาโดยกระบวนการฟื้นฟูแหล่งเรียนรู้...,232102,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,1/10/2558,30/9/2559,โครงการ,ไม่เปลี่ยนแปลง,๔.๑ การพัฒนา หมายถึง การเจริญเติบโต (Growth) เ...,๖.๑ เพื่อฝึกอบรมเชิงปฏิบัติการให้นักเรียนเป็นเ...,",สาขาสังคมวิทยา",ผู้ประสานคณะ สถาบันสำนัก กำลังตรวจสอบ,0,มหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาลัย,2019-10-31,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,yes,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,,eng,531670
5,nrms,nrms,170318,1,2559A10402040,การออกแบบเจนเนอเรเตอร์ขนาดเมโส,170318,นายJames C. Moran,1/10/2558,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"เครื่องกำเนิดพลังงานขนาดเมโส,การเผาไหม้ขนาดเมโ...",1) Design and build a meso vaporization system...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยเชียงใหม่,2019-10-31,นายJames C. Moran,นายJames C. Moran,yes,James C. Moran,นาย,eng,532125
6,nrms,nrms,169897,1,2559A30602066,ผลกระทบของหย่อมป่าและพื้นที่ชายขอบป่าต่ออัตราก...,169897,นายAndrew Pierce,1/10/2558,30/9/2560,โครงการ,เปลี่ยนแปลงมาก,"การอนุรักษ์และจัดการความหลากหลายทางชีวภาพ,การล...",1. To determine the nesting patterns of birds ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-31,นายAndrew Pierce,นายAndrew Pierce,yes,Andrew Pierce,นาย,eng,529341
7,nrms,nrms,171182,1,2559A30402137,การจัดเรียงเพนตะควาร์กของบาริออนมวลเบาที่ไม่มี...,171182,นายYupeng Yan,1/10/2557,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ควาร์ก,มีซอน,บาริออน,โครงสร้างของบาริออน,เพนตะ...",To construct the wave function of three-quark ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-31,นายYupeng Yan,นายYupeng Yan,yes,Yupeng Yan,นาย,eng,532273
8,nrms,nrms,12936,1,,การถนอมอาหารด้วยกระบวนการแบบไม่ใช้ความร้อนเพื่...,12936,Prof. Dr. โมฮัมหมัดนากิ เอชิอากิ,1/10/2561,30/9/2562,โครงการ,,"การเก็บรักษาน้ำผลไม้, กระบวนการไม่ใช้ความร้อน,...",First year: Laboratory scale Extraction of ant...,,ส่งกลั

In [ ]:
result_other

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,11775,1,,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,11775,,1/10/2561,30/9/2563,โครงการ,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,Training and workshop to increase TB research ...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no,534308
1,nrms,nrms,12698,1,,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,12698,,1/10/2560,30/9/2562,โครงการ,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,เพื่อพัฒนาชุดควบคุมการปรับระดับเมนิสคัสแบบอัตโ...,,โครงการอยู่ระหว่างดำเนินการ,0,กรมวิทยาศาสตร์บริการ,2019-10-24,,,other,no,534308
2,nrms,nrms,16150,1,,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,16150,,1/10/2561,30/9/2563,โครงการ,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",1) เพื่อประเมินโครงสร้างพื้นฐานของห้องปฏิบัติก...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no,534308
3,nrms,nrms,16384,1,,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,16384,,1/11/2561,30/7/2562,โครงการ,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,๑. เพื่อศึกษาสภาพปัญหาของมาตรการปฏิบัติต่อผู้ก...,,โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานกิจการยุติธรรม,2019-10-24,,,other,no,534308
4,nrms,nrms,17505,1,,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,17505,,1/10/2561,30/9/2565,โครงการ,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",4.1 เพื่อพัฒนาเทคนิควิธีการทดสอบและสอบเทียบที่...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,สถาบันวิจัยวิทยาศาสตร์และเทคโนโลยีแห่งประเทศไทย,2019-10-24,,,other,no,534308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,nrms,nrms,5934,1,,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,5934,,,,โครงการ,โครงการที่เข้าใหม่,,เพื่อการพัฒนาการวิเคราะห์ปริมาณ C-14 ที่นำไปสู...,,โครงการอยู่ระหว่างดำเนินการ,0,สถาบันเทคโนโลยีนิวเคลียร์แห่งชาติ (องค์การมหาชน),2019-10-24,,,other,no,534308
702,nrms,nrms,271519,1,2560A00602003,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,271519,,1/10/2559,30/9/2560,โครงการ,ไม่เปลี่ยนแปลง,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",เพื่อพัฒนาการเพาะเลี้ยงปลากระแห เพื่อการอนุรัก...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานคณะกรรมการการอาชีวศึกษา,2019-10-31,,,other,no,534308
703,nrms,nrms,626770,1,2559NRCT72022,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,626770,,1/10/2558,30/9/2559,โครงการ,โครงการที่เข้าใหม่,,สำรวจ รวบรวมความหลากหลายทางชีวภาพของสายพันธุ์พ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการวิจัยเสร็จสิ้น,0,กองบริหารแผนและงบประมาณการวิจัย (กบง.),2019-11-01,,,other,no,534308
704,nrms,nrms,544939,1,,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,544939,,1/10/2559,31/1/2561,โครงการ,ไม่เปลี่ยนแปลง,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,เพื่อหนุนเสริมและเพิ่มขีดความสามารถของเกษตรกร ...,,โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,,,other,no,534308


In [ ]:
result_all

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes,534267
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes,534268
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes,534269
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes,534270
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes,534271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27310,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes,547924
27311,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes,536246
27312,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes,541420
27313,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes,536886


# Create Publish

## Create Publish thai

In [ ]:
result_thai.columns

Index(['_index', '_type', '_id', '_score', '_source.link', '_source.name',
       '_source.id', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', '_source.word', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate', 'new_name', 'first_name',
       'language', 'first_author', 'id'],
      dtype='object')

In [ ]:
result_thai = result_thai.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [ ]:
result_thai

,_index,_type,source_ID,_score,citation,paperTitle,paper_ID,_source.author,_source.start,_source.stop,_source.kind,_source.group,paperKeywords,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,firstAuthor,auth_ID
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes,534267
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes,534268
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes,534269
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes,534270
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes,534271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26543,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes,547924
26544,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes,536246
26545,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes,541420
26546,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes,536886


In [ ]:
result_thai.columns

Index(['_index', '_type', 'source_ID', '_score', 'citation', 'paperTitle',
       'paper_ID', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', 'paperKeywords', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate', 'new_name', 'first_name',
       'language', 'firstAuthor', 'auth_ID'],
      dtype='object')

In [ ]:
result_thai = result_thai.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language','clean_name','predix'],axis=1)

In [ ]:
result_thai

,source_ID,citation,paperTitle,paper_ID,paperKeywords,firstAuthor,auth_ID
0,7072,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,yes,534267
1,7073,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,yes,534268
2,6712,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",yes,534269
3,7087,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",yes,534270
4,9502,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",yes,534271
...,...,...,...,...,...,...,...
26543,640818,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,yes,547924
26544,303080,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,yes,536246
26545,303082,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,yes,541420
26546,383660,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,,yes,536886


In [ ]:
result_thai['sourceName'] = 'nrms'
result_thai['paperType'] = 'project'

# result_thai['paper_ID']=''
# result_thai['paperTitle']=''
result_thai['paperAbtract_th'] = ''
result_thai['paperAbtract_en'] = ''
# result_thai['paperKeywords'] = ''
result_thai['paperPage'] = ''
result_thai['publishedDate'] = ''
result_thai['publishedYear'] = ''
result_thai['publisherName'] = ''
result_thai['bookName'] = ''
result_thai['bookNo'] = ''
result_thai['bookVolume'] = ''

result_thai['bookEditor'] = ''
result_thai['bookSeries'] = ''
result_thai['Fund'] = ''
result_thai['DOI'] = ''
result_thai['ISSN'] = ''
# result_thai['citation'] = ''
result_thai['refURL']=''
result_thai['crossRef'] = ''

In [ ]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [ ]:
len(result_thai.columns)

25

In [ ]:
result_thai = result_thai[all_column]

In [ ]:
result_thai

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534267,yes,nrms,7072,7072,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,project,,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,,,,,,,,,,,,,,,
1,534268,yes,nrms,7073,7073,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,project,,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,,,,,,,,,,,,,,,
2,534269,yes,nrms,6712,6712,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,project,,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",,,,,,,,,,,,,,,
3,534270,yes,nrms,7087,7087,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,project,,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",,,,,,,,,,,,,,,
4,534271,yes,nrms,9502,9502,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,project,,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26543,547924,yes,nrms,640818,640818,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,project,,,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,,,,,,,,,,,,,2558000013130,,
26544,536246,yes,nrms,303080,303080,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,project,,,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,,,,,,,,,,,,,2558000111024,,
26545,541420,yes,nrms,303082,303082,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,project,,,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,,,,,,,,,,,,,2558000112271,,
26546,536886,yes,nrms,383660,383660,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,project,,,,,,,,,,,,,,,,2558000112589,,


## Create Publish ENG

In [ ]:
result_eng = result_eng.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [ ]:
result_eng = result_eng.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language','clean_name','predix'],axis=1)

In [ ]:
result_eng['sourceName'] = 'nrms'
result_eng['paperType'] = 'project'

# result_eng['paper_ID']=''
# result_eng['paperTitle']=''
result_eng['paperAbtract_th'] = ''
result_eng['paperAbtract_en'] = ''
# result_eng['paperKeywords'] = ''
result_eng['paperPage'] = ''
result_eng['publishedDate'] = ''
result_eng['publishedYear'] = ''
result_eng['publisherName'] = ''
result_eng['bookName'] = ''
result_eng['bookNo'] = ''
result_eng['bookVolume'] = ''

result_eng['bookEditor'] = ''
result_eng['bookSeries'] = ''
result_eng['Fund'] = ''
result_eng['DOI'] = ''
result_eng['ISSN'] = ''
# result_eng['citation'] = ''
result_eng['refURL']=''
result_eng['crossRef'] = ''

In [ ]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [ ]:
len(result_thai.columns)

25

In [ ]:
result_eng = result_eng[all_column]

In [ ]:
result_eng.head()

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534565,yes,nrms,29416,29416,การใช้โปรไบโอติกที่พบในปลาเพื่อเป็นทางเลือกแทน...,project,,,"โพรไบโอติก, ปลานิล, Streptococcus agalactiae",,,,,,,,,,,,,,,
1,534566,yes,nrms,29419,29419,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,project,,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",,,,,,,,,,,,,,,
2,535120,yes,nrms,2006,2006,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,project,,,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,,,,,,,,,,,,,256109A3060066,,
3,536305,yes,nrms,254239,254239,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,project,,,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",,,,,,,,,,,,,2560A30402173,,
4,536409,yes,nrms,255170,255170,การพัฒนาวัสดุเคลือบอัลลอยคอมพอสิตโลหะผสมนิกเกิ...,project,,,"เคลือบนิกเกิล-ทังสเตน-โบรอน,การชุบไฟฟ้าแบบร่วม...",,,,,,,,,,,,,2560A10102005,,


## Create Publish Other

In [ ]:
result_other = result_other.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [ ]:
result_other = result_other.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language','clean_name','predix'],axis=1)

In [ ]:
result_other['sourceName'] = 'nrms'
result_other['paperType'] = 'project'

# result_other['paper_ID']=''
# result_other['paperTitle']=''
result_other['paperAbtract_th'] = ''
result_other['paperAbtract_en'] = ''
# result_other['paperKeywords'] = ''
result_other['paperPage'] = ''
result_other['publishedDate'] = ''
result_other['publishedYear'] = ''
result_other['publisherName'] = ''
result_other['bookName'] = ''
result_other['bookNo'] = ''
result_other['bookVolume'] = ''

result_other['bookEditor'] = ''
result_other['bookSeries'] = ''
result_other['Fund'] = ''
result_other['DOI'] = ''
result_other['ISSN'] = ''
# result_other['citation'] = ''
result_other['refURL']=''
result_other['crossRef'] = ''

In [ ]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [ ]:
len(result_other.columns)

25

In [ ]:
result_other = result_other[all_column]

In [ ]:
result_other

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534308,no,nrms,11775,11775,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,project,,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,,,,,,,,,,,,,,,
1,534308,no,nrms,12698,12698,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,project,,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,,,,,,,,,,,,,,,
2,534308,no,nrms,16150,16150,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,project,,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",,,,,,,,,,,,,,,
3,534308,no,nrms,16384,16384,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,project,,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,,,,,,,,,,,,,,,
4,534308,no,nrms,17505,17505,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,project,,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,534308,no,nrms,5934,5934,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,project,,,,,,,,,,,,,,,,,,
702,534308,no,nrms,271519,271519,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,project,,,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",,,,,,,,,,,,,2560A00602003,,
703,534308,no,nrms,626770,626770,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,project,,,,,,,,,,,,,,,,2559NRCT72022,,
704,534308,no,nrms,544939,544939,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,project,,,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,,,,,,,,,,,,,,,


## Create Publish ALL

In [ ]:
result_all = result_all.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [ ]:
result_all = result_all.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language','clean_name','predix'],axis=1)

In [ ]:
result_all['sourceName'] = 'nrms'
result_all['paperType'] = 'project'

# result_all['paper_ID']=''
# result_all['paperTitle']=''
result_all['paperAbtract_th'] = ''
result_all['paperAbtract_en'] = ''
# result_all['paperKeywords'] = ''
result_all['paperPage'] = ''
result_all['publishedDate'] = ''
result_all['publishedYear'] = ''
result_all['publisherName'] = ''
result_all['bookName'] = ''
result_all['bookNo'] = ''
result_all['bookVolume'] = ''

result_all['bookEditor'] = ''
result_all['bookSeries'] = ''
result_all['Fund'] = ''
result_all['DOI'] = ''
result_all['ISSN'] = ''
# result_all['citation'] = ''
result_all['refURL']=''
result_all['crossRef'] = ''

In [ ]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [ ]:
len(result_all.columns)

25

In [ ]:
result_all = result_all[all_column]

In [ ]:
result_all

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534267,yes,nrms,7072,7072,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,project,,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,,,,,,,,,,,,,,,
1,534268,yes,nrms,7073,7073,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,project,,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,,,,,,,,,,,,,,,
2,534269,yes,nrms,6712,6712,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,project,,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",,,,,,,,,,,,,,,
3,534270,yes,nrms,7087,7087,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,project,,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",,,,,,,,,,,,,,,
4,534271,yes,nrms,9502,9502,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,project,,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27310,547924,yes,nrms,640818,640818,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,project,,,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,,,,,,,,,,,,,2558000013130,,
27311,536246,yes,nrms,303080,303080,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,project,,,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,,,,,,,,,,,,,2558000111024,,
27312,541420,yes,nrms,303082,303082,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,project,,,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,,,,,,,,,,,,,2558000112271,,
27313,536886,yes,nrms,383660,383660,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,project,,,,,,,,,,,,,,,,2558000112589,,


## df_author

In [ ]:
df_unique_name = df_unique_name.rename(columns={"clean_name": "authFullName"})

In [ ]:
df_unique_name['language'] = df_unique_name['authFullName'].apply(language)

In [ ]:
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

In [ ]:
result_author_thai

,new_name,id,language
0,นายศิวกร อ่างทอง,534267,th
1,ผศ. อำนวย เรืองวารี,534268,th
2,นายประสิทธิ์ ศรีนคร,534269,th
3,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,534270,th
4,รศ. ดร. จตุรงค์ ลังกาพินธุ์,534271,th
...,...,...,...
13654,นายจักรกฤษณ์ พจนศิลป์,547921,th
13655,นายเติมเพชร สุขคณาภิบาล,547922,th
13656,นางนัทธีรัตน์ พีระพันธุ์,547923,th
13657,นายอดิศักดิ์ จำปาทอง,547924,th


In [ ]:
result_author_eng

,new_name,id,language
298,Assist. Prof. Dr.,534565,eng
299,Dr. Hien Van Doan,534566,eng
853,นายAndrew Pierce,535120,eng
2038,Prof.Dr. James Ketudat-Cairns,536305,eng
2142,Dr. เจียเชียน ฉิน,536409,eng
2746,Dr. กรวรรณ ศรีงาม,537013,eng
3182,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,537449,eng
3637,นายJames C. Moran,537904,eng
3785,นายYupeng Yan,538052,eng
4364,Dr.,538631,eng


In [ ]:
result_author_other

,new_name,id,language
41,,534308,other


# Export

In [ ]:
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

result_thai.index.name = 'id'
result_eng.index.name = 'id'
result_other.index.name = 'id'
result_all.index.name = 'id'

result_thai.to_csv('nrms_publish_thai.csv')
result_eng.to_csv('nrms_publish_eng.csv')
result_other.to_csv('nrms_publish_other.csv')
result_all.to_csv('nrms_publish_all.csv')

result_author_thai[['id','authFullName']].to_csv('nrms_author_thai.csv')
result_author_eng[['id','authFullName']].to_csv('nrms_author_eng.csv')
result_author_other[['id','authFullName']].to_csv('nrms_author_other.csv')